<a href="https://colab.research.google.com/github/RadjaOurida/fraudDetection/blob/main/Copie_de_pr%C3%A9traitement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
%matplotlib inline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_train_Tr=pd.read_csv("/content/drive/My Drive/ML_Projet_MCNS/train_transaction.csv")
df_train_identity=pd.read_csv("/content/drive/My Drive/ML_Projet_MCNS/train_identity.csv")
df_test_id=pd.read_csv("/content/drive/My Drive/ML_Projet_MCNS/test_identity.csv")
df_test_Tr=pd.read_csv("/content/drive/My Drive/ML_Projet_MCNS/test_transaction.csv")

In [ ]:
train= df_train_Tr.merge(df_train_identity, how='left', on="TransactionID")
test = df_test_Tr.merge(df_test_id, how='left', on="TransactionID")

In [ ]:
print("The shape of train is: %s, %s" % (train.shape[0],train.shape[1]))
print("The shape of test is: %s, %s" % (test.shape[0],test.shape[1]))

The shape of train is: 590540, 434
The shape of test is: 506691, 433


In [ ]:
test = test.rename(columns={"id-01": "id_01", "id-02": "id_02", "id-03": "id_03", 
                            "id-06": "id_06", "id-05": "id_05", "id-04": "id_04", 
                            "id-07": "id_07", "id-08": "id_08", "id-09": "id_09", 
                            "id-10": "id_10", "id-11": "id_11", "id-12": "id_12", 
                            "id-15": "id_15", "id-14": "id_14", "id-13": "id_13", 
                            "id-16": "id_16", "id-17": "id_17", "id-18": "id_18", 
                            "id-21": "id_21", "id-20": "id_20", "id-19": "id_19", 
                            "id-22": "id_22", "id-23": "id_23", "id-24": "id_24", 
                            "id-27": "id_27", "id-26": "id_26", "id-25": "id_25", 
                            "id-28": "id_28", "id-29": "id_29", "id-30": "id_30", 
                            "id-31": "id_31", "id-32": "id_32", "id-33": "id_33", 
                            "id-34": "id_34", "id-35": "id_35", "id-36": "id_36", 
                            "id-37": "id_37", "id-38": "id_38"})

### ***Create new feature from TransactionDT***

In [ ]:
train['hour'] = ((train['TransactionDT']//3600)%24)+1
test['hour'] = ((test['TransactionDT']//3600)%24)+1
train

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,hour
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,debit,272.0,87.0,48.0,NaN,NaN,NaN,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,2.0,29.0,29.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,debit,204.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,debit,231.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,debit,387.0,87.0,3.0,NaN,aol.com,NaN,1.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,1.0,1.0,5.0,1.0,22.0,22.0,0.0,22.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

## **Card3**

In [ ]:
def new_card3(row):
    if row==np.nan:
        return row
    else:
        if row > 160 :
            return 'Positive'
        else:
            return 'Negative'
        
train['card3'] = train['card3'].apply(new_card3)
test['card3'] = test['card3'].apply(new_card3)


**card6** *Type of card*

In [ ]:
def replacetodebit(row):
    if row==np.nan:
        return row
    if row=='debit or credit' or row=='charge card':
        return 'debit'
    else:
        return row
    
train['card6'] = train['card6'].apply(replacetodebit)
test['card6'] = test['card6'].apply(replacetodebit)

### **P_emaildomain R_emaildomain**

In [ ]:
# Reference: https://www.kaggle.com/kabure/extensive-eda-and-modeling-xgb-hyperopt/notebook#Ploting-Transaction-Amount-Values-Distribution
train.loc[train['P_emaildomain'].isin(['gmail.com', 'gmail']),'P_emaildomain'] = 'Google'
train.loc[train['P_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                         'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                         'yahoo.es']), 'P_emaildomain'] = 'Yahoo'
train.loc[train['P_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                         'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                         'outlook.es', 'live.com', 'live.fr',
                                         'hotmail.fr']), 'P_emaildomain'] = 'Microsoft'
train.loc[train.P_emaildomain.isin(train.P_emaildomain.value_counts()[train.P_emaildomain.value_counts() <= 500 ].index), 'P_emaildomain'] = "Others"
train.P_emaildomain.fillna("NoInf", inplace=True)

train.loc[train['R_emaildomain'].isin(['gmail.com', 'gmail']),'R_emaildomain'] = 'Google'

train.loc[train['R_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                             'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                             'yahoo.es']), 'R_emaildomain'] = 'Yahoo'
train.loc[train['R_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                             'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                             'outlook.es', 'live.com', 'live.fr',
                                             'hotmail.fr']), 'R_emaildomain'] = 'Microsoft'
train.loc[train.R_emaildomain.isin(train.R_emaildomain.value_counts()[train.R_emaildomain.value_counts() <= 300 ].index), 'R_emaildomain'] = "Others"
train.R_emaildomain.fillna("NoInf", inplace=True)

In [ ]:
test.loc[test['P_emaildomain'].isin(['gmail.com', 'gmail']),'P_emaildomain'] = 'Google'
test.loc[test['P_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                         'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                         'yahoo.es']), 'P_emaildomain'] = 'Yahoo'
test.loc[test['P_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                         'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                         'outlook.es', 'live.com', 'live.fr',
                                         'hotmail.fr']), 'P_emaildomain'] = 'Microsoft'
test.loc[test.P_emaildomain.isin(test.P_emaildomain.value_counts()[test.P_emaildomain.value_counts() <= 500 ].index), 'P_emaildomain'] = "Others"
test.P_emaildomain.fillna("NoInf", inplace=True)

test.loc[test['R_emaildomain'].isin(['gmail.com', 'gmail']),'R_emaildomain'] = 'Google'

test.loc[test['R_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                             'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                             'yahoo.es']), 'R_emaildomain'] = 'Yahoo'
test.loc[test['R_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                             'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                             'outlook.es', 'live.com', 'live.fr',
                                             'hotmail.fr']), 'R_emaildomain'] = 'Microsoft'
test.loc[test.R_emaildomain.isin(test.R_emaildomain.value_counts()[test.R_emaildomain.value_counts() <= 300 ].index), 'R_emaildomain'] = "Others"
test.R_emaildomain.fillna("NoInf", inplace=True)

In [ ]:
train.drop(columns={"TransactionDT"},inplace=True)
test.drop(columns={"TransactionDT"},inplace=True)


# **Prétraiement**

### **Identic Columns**

In [ ]:
def get_rate_IV(df_train_complete):
  nbr=df_train_complete.shape[0]
  df_col_elim =pd.DataFrame()
  for i in range(df_train_complete.shape[1]):
    max_rate_IV= ((df_train_complete.iloc[:,i].value_counts()).max())/nbr
    df_col_elim=df_col_elim.append({"Column":df_train_complete.columns[i],"Rate_M_IV":max_rate_IV},ignore_index=True)##
  return df_col_elim

def eliminate_rate_IV(df_train_complete,df_test_complete):
  df_col_elim = get_rate_IV(df_train_complete)
  #print(df_col_elim[df_col_elim["Rate_M_IV" ]> 0.99])
  df_col_elim2= get_rate_IV(df_test_complete)
  #print(df_col_elim2[df_col_elim2["Rate_M_IV" ]> 0.99])
  int_df = pd.merge(df_col_elim[df_col_elim["Rate_M_IV" ]> 0.99], df_col_elim2[df_col_elim2["Rate_M_IV" ]> 0.99], how ='inner', on =["Column"]) 
  print(int_df)
  df_train_complete.drop(int_df.Column, axis=1, inplace=True)
  df_test_complete.drop(int_df.Column, axis=1, inplace=True)

eliminate_rate_IV(train,test)


   Column  Rate_M_IV_x  Rate_M_IV_y
0    V107     0.999048     1.000000
1    V108     0.994415     0.993643
2    V111     0.996544     0.996438
3    V112     0.994400     0.992824
4    V113     0.995983     0.995451
5    V117     0.998237     0.999045
6    V118     0.997795     0.997697
7    V119     0.998154     0.998540
8    V120     0.997761     0.998234
9    V121     0.995011     0.993714
10   V122     0.997131     0.997085
11   V305     0.999973     0.999992


## **Missing Values **

In [ ]:
def detectmissing(df):
    summary = pd.DataFrame(df.dtypes,columns=['dtypes']) # (the data, the name of column, the index set them by his own)
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']] #we ignore the column "index"
    summary['Missing(%)'] = (df.isnull().sum().values / df.shape[0])*100  
    #summary['Uniques'] = df.nunique().values
    return summary

In [ ]:
summary_train=detectmissing(train)

In [ ]:
summary_test = detectmissing(test)

### **Delete the columns with rate of missing data >=80**

In [ ]:
def delete_col_80(df_train_complete,df_test_complete,summary_train):
  col_80= summary_train[summary_train["Missing(%)"]>= 80]["Name"].tolist()
  df_train_complete.drop(col_80, axis=1, inplace=True)
  df_test_complete.drop(col_80, axis=1, inplace=True)


delete_col_80(train,test,summary_train)

### **Impute Missing data**

In [ ]:
def imputeDM (train, test):
  Y = train.isFraud  
  train.drop(columns = ['isFraud'],inplace=True) 
  cat_cols = train.select_dtypes(include = 'object').columns 
  for i in cat_cols:
    train[i] = train[i].fillna(train[i].mode()[0])
    test[i] = test[i].fillna(train[i].mode()[0])

  num_cols = train.select_dtypes(include=np.number).columns 
  for i in num_cols:
    train[i] = train[i].fillna(train[i].median())
    test[i] = test[i].fillna(train[i].median())

  train['isFraud'] = Y
  return (train, test)

train, test = imputeDM (train, test)
       

# **Outliers**

In [ ]:
print(train[train["isFraud"]==1 ].shape)

(20663, 348)


In [ ]:
train.drop(train[train.TransactionAmt>10000].index, axis=0, inplace=True)
train.drop(train[train.dist1>6000].index, axis=0, inplace=True)

train.loc[train.C1> 3000, "C1" ] = 3000
train.loc[train.C2> 3000, "C2" ] = 3000
train.loc[train.C4 > 1600, "C4" ] = 1600
train.loc[train.C6> 1600, "C6" ] = 1600
train.loc[train.C7 > 1600, "C7" ] = 1600
train.loc[train.C10 > 2000 , "C10" ] = 2000
train.loc[train.C11 > 2000  , "C11" ]= 2000
train.loc[train.C12> 2000, "C12" ] = 2000
train.loc[train.C13 > 2000 , "C13" ] = 2000
train.loc[train.C14> 800, "C14" ] = 800



In [ ]:
print(train[train["isFraud"]==1 ].shape)

(20663, 348)


*** labelEncoder***

---






In [ ]:
def LabelEncode(train,test):
  encoders = dict() ##La fonction dict () crée un dictionnaire.
  cat_cols = train.select_dtypes(include = 'object').columns
  for col_name in cat_cols :
    series_train = train[col_name]
    series_test = test[col_name]

    label_encoder = LabelEncoder()
    label_encoder.fit(list(series_train.astype(str).values) + list(series_test.astype(str).values))
        
    train[col_name] = pd.Series(
    label_encoder.transform(series_train),
    index=series_train.index
    )
    test[col_name] = pd.Series(
    label_encoder.transform(series_test),
    index=series_test.index
    )
  return (train, test)

train, test=LabelEncode(train,test)

## **Normalisation**

In [ ]:
from sklearn.preprocessing import minmax_scale

def normalize (train, test):
  cols = test.columns.tolist()
  cols.remove("TransactionID")

  for col in cols:
    train[col] = (minmax_scale(train[col]))  #normalisation scaling ordre de grandeur 
    test[col] = (minmax_scale(test[col]) )
  #return (train, test)

#train , test = 
normalize (train, test)

In [ ]:
train

,TransactionID,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,...,V304,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,hour,isFraud
0,2987000,0.010580,1.00,0.743044,0.522,0.0,0.333333,0.306569,0.0,0.488636,0.836957,0.003498,0.10,0.142857,0.000333,0.000333,0.0,0.0,0.000000,0.000625,0.0,0.0000,0.004762,0.0000,0.0010,0.0000,0.0005,0.00125,0.021875,0.151563,0.015873,0.149344,0.01221,0.014840,0.091286,0.086279,1.0,1.0,1.0,1.0,...,0.0000,0.000000,0.000803,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000873,0.000000,0.00000,0.00000,0.00000,0.95,0.125851,0.580645,1.0,1.0,1.0,0.777778,0.0,0.0,0.511628,0.422067,0.663102,0.0,0.0,0.28655,1.0,0.0,1.0,0.0,0.0,0.902788,0.0,0
1,2987001,0.004457,1.00,0.100885,0.608,0.0,0.666667,0.014599,0.0,0.511364,0.836957,0.001473,0.00,0.142857,0.000333,0.000333,0.0,0.0,0.000000,0.000625,0.0,0.0000,0.000000,0.0000,0.0005,0.0000,0.0005,0.00125,0.000000,0.151563,0.009768,0.123108,0.01221,0.000000,0.132780,0.086279,1.0,1.0,1.0,0.0,...,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.95,0.125851,0.580645,1.0,1.0,1.0,0.777778,0.0,0.0,0.511628,0.422067,0.663102,0.0,0.0,0.28655,1.0,0.0,1.0,0.0,0.0,0.902788,0.0,0
2,2987002,0.009107,1.00,0.210566,0.780,0.0,1.000000,0.481752,1.0,0.522727,0.836957,0.052845,0.05,0.142857,0.000333,0.000333,0.0,0.0,0.000000,0.000625,0.0,0.0000,0.004762,0.0000,0.0005,0.0000,0.0005,0.00125,0.000000,0.151563,0.009768,0.123108,0.01221,0.000000,0.508990,0.413721,1.0,1.0,1.0,0.0,...,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.95,0.125851,0.580645,1.0,1.0,1.0,0.777778,0.0,0.0,0.511628,0.422067,0.663102,0.0,0.0,0.28655,1.0,0.0,1.0,0.0,0.0,0.902788,0.0,0
3,2987003,0.007712,1.00,0.984824,0.934,0.0,0.666667,0.124088,1.0,0.854545,0.836957,0.001473,0.20,0.142857,0.000667,0.001667,0.0,0.0,0.000000,0.002500,0.0,0.0000,0.004762,0.0000,0.0005,0.0000,0.0125,0.00125,0.175000,0.175000,0.000000,0.217962,0.00000,0.095890,0.132780,0.201663,1.0,1.0,1.0,0.0,...,0.0000,0.000460,0.012061,0.008502,0.000000,0.006422,0.0,0.002449,0.000000,0.000000,0.000000,0.000533,0.010476,0.008022,0.00000,0.00000,0.00000,0.95,0.125851,0.580645,1.0,1.0,1.0,0.777778,0.0,0.0,0.511628,0.422067,0.663102,0.0,0.0,0.28655,1.0,0.0,1.0,0.0,0.0,0.902788,0.0,0
4,2987004,0.007712,0.25,0.201023,0.828,0.0,0.666667,0.014599,0.0,0.727273,0.836957,0.001473,0.00,0.142857,0.000333,0.000333,0.0,0.0,0.000000,0.000625,0.0,0.0003,0.000000,0.0005,0.0005,0.0000,0.0005,0.00125,0.000000,0.151563,0.009768,0.149344,0.01221,0.017123,0.132780,0.140333,1.0,1.0,1.0,0.0,...,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,1.00,0.070815,0.580645,1.0,1.0,1.0,0.777778,0.5,1.0,0.511628,0.774081,0.078431,1.0,1.0,0.94152,1.0,0.0,1.0,1.0,1.0,0.559328,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0.007557,1.00,0.319039,0.522,0.0,1.000000,0.919708,1.0,0.390909,0.836957,0.008838,0.10,0.142857,0.000667,0.000333,0.0,0.0,0.002865,0.000000,0.0,0.0000,0.009524,0.0000,0.0005,0.0000,0.0015,0.00250,0.045313,0.045313,0.036630,0.149344,0.01221,0.063927,0.150761,0.144491,1.0,1.0,1.0,0.0,...,0.0000,0.000000,0.000329,0.000000,0.000000,0.000870,0.0,0.000000,0.009953,

# **Réduire la mémoire**

In [ ]:
# credit to @guiferviz for the memory reduction 
def memory_usage_mb(df, *args, **kwargs):
    """Dataframe memory usage in MB. """
    return df.memory_usage(*args, **kwargs).sum() / 1024**2

def reduce_memory_usage(df, deep=True, verbose=True):
    # All types that we want to change for "lighter" ones.
    # int8 and float16 are not include because we cannot reduce
    # those data types.
    # float32 is not include because float16 has too low precision.
    numeric2reduce = ["int16", "int32", "int64", "float64"]
    start_mem = 0
    if verbose:
        start_mem = memory_usage_mb(df, deep=deep)

    for col, col_type in df.dtypes.iteritems():
        best_type = None
        if col_type in numeric2reduce:
            downcast = "integer" if "int" in str(col_type) else "float"
            df[col] = pd.to_numeric(df[col], downcast=downcast)
            best_type = df[col].dtype.name
        # Log the conversion performed.
        if verbose and best_type is not None and best_type != str(col_type):
            print(f"Column '{col}' converted from {col_type} to {best_type}")

    if verbose:
        end_mem = memory_usage_mb(df, deep=deep)
        diff_mem = start_mem - end_mem
        percent_mem = 100 * diff_mem / start_mem
        #print(f"Memory usage decreased from"
        #      f" {start_mem:.2f}MB to {end_mem:.2f}MB"
        #      f" ({diff_mem:.2f}MB, {percent_mem:.2f}% reduction)")
        
    return df

In [ ]:
train = reduce_memory_usage(train)
test = reduce_memory_usage(test) 

Column 'TransactionID' converted from int64 to int32
Column 'TransactionAmt' converted from float64 to float32
Column 'ProductCD' converted from float64 to float32
Column 'card1' converted from float64 to float32
Column 'card2' converted from float64 to float32
Column 'card3' converted from float64 to float32
Column 'card4' converted from float64 to float32
Column 'card5' converted from float64 to float32
Column 'card6' converted from float64 to float32
Column 'addr1' converted from float64 to float32
Column 'addr2' converted from float64 to float32
Column 'dist1' converted from float64 to float32
Column 'P_emaildomain' converted from float64 to float32
Column 'R_emaildomain' converted from float64 to float32
Column 'C1' converted from float64 to float32
Column 'C2' converted from float64 to float32
Column 'C3' converted from float64 to float32
Column 'C4' converted from float64 to float32
Column 'C5' converted from float64 to float32
Column 'C6' converted from float64 to float32
Colum

# **ACP**

In [ ]:
train.columns.to_list()

['TransactionID',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'P_emaildomain',
 'R_emaildomain',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D10',
 'D11',
 'D15',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V40',
 'V41',
 'V42',
 'V43',
 'V44',
 'V45',
 'V46',
 'V47',
 'V48',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V65',
 'V66',
 'V67',
 'V68',
 'V69',
 'V70',
 'V71',
 'V72',
 'V73',
 'V74',
 'V75',
 'V76',

In [ ]:
train.reset_index()
Vxx = train.loc[:,"V1":"V321"]  
Vxx

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,...,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,1.0,0.125,0.111111,0.166667,0.166667,0.111111,0.111111,0.125,0.125,0.00,0.0,0.333333,0.166667,1.0,0.0,0.0,0.0,0.0,0.142857,0.066667,0.0,0.0,0.076923,0.076923,0.142857,0.076923,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.2,0.018519,0.018519,0.0,0.0,...,0.0,0.03125,0.014706,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.0000,0.000000,0.000803,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000873,0.000000,0.00000,0.00000,0.00000
1,1.0,0.125,0.111111,0.166667,0.166667,0.111111,0.111111,0.125,0.125,0.00,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.142857,0.066667,0.0,0.0,0.076923,0.076923,0.142857,0.076923,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.018519,0.018519,0.0,0.0,...,0.0,0.03125,0.014706,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000
2,1.0,0.125,0.111111,0.166667,0.166667,0.111111,0.111111,0.125,0.125,0.00,0.0,0.333333,0.166667,1.0,0.0,0.0,0.0,0.0,0.142857,0.066667,0.0,0.0,0.076923,0.076923,0.142857,0.076923,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.2,0.018519,0.018519,0.0,0.0,...,0.0,0.03125,0.014706,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000
3,1.0,0.125,0.111111,0.166667,0.166667,0.111111,0.111111,0.125,0.125,0.00,0.0,0.333333,0.166667,1.0,0.0,0.0,0.0,0.0,0.142857,0.066667,0.0,0.0,0.076923,0.076923,0.142857,0.076923,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.2,0.018519,0.018519,0.0,0.0,...,0.0,0.00000,0.000000,0.000000,0.105263,0.0,0.129032,0.0,0.000000,0.0,0.000000,0.0,0.001151,0.029549,0.025862,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.0000,0.000460,0.012061,0.008502,0.000000,0.006422,0.0,0.002449,0.000000,0.000000,0.000000,0.000533,0.010476,0.008022,0.00000,0.00000,0.00000
4,1.0,0.125,0.111111,0.166667,0.166667,0.111111,0.111111,0.125,0.125,0.00,0.0,0.333333,0.166667,1.0,0.0,0.0,0.0,0.0,0.142857,0.066667,0.0,0.0,0.076923,0.076923,0.142857,0.076923,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.2,0.018519,0.018519,0.0,0.0,...,0.0,0.03125,0.014706,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.05,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,1.0,0.125,0.111111,0.000000,0.000000,0.111111,0.111111,0.125,0.125,0.00,0.0,0.666667,0.333333,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.076923,0.076923,0.142857,0.076923,0.0,0.0,0.2,0.111111,0.0,0.0,0.0,0.000000,0.333333,0.2,0.018519,0.018519,0.0,0.0,...,0.0,0.03125,0.014706,0.000000,0.010526,0.0,0.000000,0.1,0.083333,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [ ]:
def PCA_change(df, cols, n_components, prefix='PCA_', rand_seed=4):
    #pca = PCA(random_state=rand_seed)
    pca = PCA(n_components=n_components, random_state=rand_seed)
    principalComponents = pca.fit_transform(df[cols])
   
   
    principalDf = pd.DataFrame(principalComponents)

    df.drop(cols, axis=1, inplace=True)

    principalDf.rename(columns=lambda x: str(prefix)+str(x), inplace=True)

    df = df.merge(principalDf,right_index =True, left_index =True)
    
    return (df,pca)


train,pca = PCA_change(train, Vxx.columns,n_components=100, prefix='PCA_V_',rand_seed=4)


In [ ]:
train

,TransactionID,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,...,PCA_V_60,PCA_V_61,PCA_V_62,PCA_V_63,PCA_V_64,PCA_V_65,PCA_V_66,PCA_V_67,PCA_V_68,PCA_V_69,PCA_V_70,PCA_V_71,PCA_V_72,PCA_V_73,PCA_V_74,PCA_V_75,PCA_V_76,PCA_V_77,PCA_V_78,PCA_V_79,PCA_V_80,PCA_V_81,PCA_V_82,PCA_V_83,PCA_V_84,PCA_V_85,PCA_V_86,PCA_V_87,PCA_V_88,PCA_V_89,PCA_V_90,PCA_V_91,PCA_V_92,PCA_V_93,PCA_V_94,PCA_V_95,PCA_V_96,PCA_V_97,PCA_V_98,PCA_V_99
0,2987000,0.010580,1.00,0.743044,0.522,0.0,0.333333,0.306569,0.0,0.488636,0.836957,0.003498,0.10,0.142857,0.000333,0.000333,0.0,0.000000,0.000000,0.000625,0.0,0.0000,0.004762,0.0000,0.0010,0.0,0.0005,0.00125,0.021875,0.151562,0.015873,0.149344,0.012210,0.014840,0.091286,0.086279,1.0,1.0,1.0,1.0,...,0.003138,-0.008753,0.005812,-0.004676,0.004567,-0.002258,-0.004045,0.007328,-0.001489,0.001058,0.000895,0.000195,0.005586,-0.002391,0.000733,0.006954,0.000233,0.004793,-0.000256,-0.001592,0.000159,0.000683,0.000204,-0.005053,0.005419,0.000204,0.005208,0.000640,0.002299,-0.001279,-0.001201,0.001974,-0.000036,-0.001525,-0.001759,0.000013,-0.000940,-0.000255,-0.001004,0.000587
1,2987001,0.004457,1.00,0.100885,0.608,0.0,0.666667,0.014599,0.0,0.511364,0.836957,0.001473,0.00,0.142857,0.000333,0.000333,0.0,0.000000,0.000000,0.000625,0.0,0.0000,0.000000,0.0000,0.0005,0.0,0.0005,0.00125,0.000000,0.151562,0.009768,0.123108,0.012210,0.000000,0.132780,0.086279,1.0,1.0,1.0,0.0,...,0.003356,0.000825,0.004826,0.001830,0.007525,-0.001530,-0.001450,0.003740,0.000917,0.000236,-0.000702,-0.000709,0.000714,-0.001599,0.000108,-0.002532,-0.000215,0.000472,-0.000332,-0.000364,0.000648,-0.000813,-0.002341,-0.001591,0.001614,0.002530,0.000751,-0.001385,0.001925,-0.001152,-0.000717,0.000732,-0.000800,-0.002315,0.000779,-0.000748,-0.000895,-0.000868,-0.000469,-0.000156
2,2987002,0.009107,1.00,0.210566,0.780,0.0,1.000000,0.481752,1.0,0.522727,0.836957,0.052845,0.05,0.142857,0.000333,0.000333,0.0,0.000000,0.000000,0.000625,0.0,0.0000,0.004762,0.0000,0.0005,0.0,0.0005,0.00125,0.000000,0.151562,0.009768,0.123108,0.012210,0.000000,0.508990,0.413721,1.0,1.0,1.0,0.0,...,0.001371,-0.001642,0.005054,0.001586,0.005869,0.000777,-0.000649,0.003530,0.002276,0.000199,-0.001134,-0.000904,-0.001181,-0.001386,0.001493,-0.001822,-0.000350,-0.000196,-0.000176,-0.000619,0.000625,-0.001257,-0.000861,-0.000560,0.000787,0.000692,0.000057,-0.000912,0.000844,-0.000906,-0.001110,0.000890,-0.000792,-0.001566,-0.000169,-0.000601,-0.000320,-0.000304,-0.000162,-0.000447
3,2987003,0.007712,1.00,0.984824,0.934,0.0,0.666667,0.124088,1.0,0.854545,0.836957,0.001473,0.20,0.142857,0.000667,0.001667,0.0,0.000000,0.000000,0.002500,0.0,0.0000,0.004762,0.0000,0.0005,0.0,0.0125,0.00125,0.175000,0.175000,0.000000,0.217962,0.000000,0.095890,0.132780,0.201663,1.0,1.0,1.0,0.0,...,0.010969,0.009994,0.007749,0.000764,0.015262,0.001130,-0.011541,0.005189,-0.004008,0.036458,0.019653,0.023859,-0.013045,0.005151,0.009450,0.000154,0.004531,0.007074,0.001041,0.002140,-0.000982,0.018605,0.014639,0.005593,-0.008508,-0.003794,-0.005007,0.001731,-0.001520,0.002764,0.005642,-0.020309,0.000787,-0.004244,-0.007078,-0.002177,0.003451,0.002884,0.011646,0.006331
4,2987004,0.007712,0.25,0.201023,0.828,0.0,0.666667,0.014599,0.0,0.727273,0.836957,0.001473,0.00,0.142857,0.000333,0.000333,0.0,0.000000,0.000000,0.000625,0.0,0.0003,0.000000,0.0005,0.0005,0.0,0.0005,0.00125,0.000000,0.151562,0.009768,0.149344,0.012210,0.017123,0.132780,0.140333,1.0,1.0,1.0,0.0,...,0.003390,0.004791,-0.009289,0.000988,-0.001484,-0.004004,-0.004092,-0.003537,-0.004441,-0.002608,0.002127,0.001623,0.001893,-0.002593,-0.000934,0.001538,0.000054,-0.000991,-0.000670,0.000090,0.000501,0.000771,0.001014,-0.001666,-0.000021,-0.000741,-0.000826,0.000221,-0.001221,0.000434,-0.000118,0.000711,0.000895,-0.000533,0.002246,0.001569,-0.000269,-0.000678,0.001081,0.001613
...,...,...,...,...,...,...,...,

In [ ]:
#train.reset_index()
Vxx = test.loc[:,"V1":"V321"]  
#Vxx
test,pca = PCA_change(test, Vxx.columns,n_components=100, prefix='PCA_V_',rand_seed=4)

In [ ]:
test

,TransactionID,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,...,PCA_V_60,PCA_V_61,PCA_V_62,PCA_V_63,PCA_V_64,PCA_V_65,PCA_V_66,PCA_V_67,PCA_V_68,PCA_V_69,PCA_V_70,PCA_V_71,PCA_V_72,PCA_V_73,PCA_V_74,PCA_V_75,PCA_V_76,PCA_V_77,PCA_V_78,PCA_V_79,PCA_V_80,PCA_V_81,PCA_V_82,PCA_V_83,PCA_V_84,PCA_V_85,PCA_V_86,PCA_V_87,PCA_V_88,PCA_V_89,PCA_V_90,PCA_V_91,PCA_V_92,PCA_V_93,PCA_V_94,PCA_V_95,PCA_V_96,PCA_V_97,PCA_V_98,PCA_V_99
0,3663549,0.003109,1.0,0.540814,0.022,0.0,1.000000,0.919708,1.0,0.159091,0.836957,0.000124,0.00,0.142857,0.002034,0.001832,0.0,0.000000,0.007979,0.002498,0.000000,0.000000,0.010490,0.000000,0.002238,0.000448,0.073624,0.007528,0.653666,0.653666,0.025093,0.364803,0.024816,0.383135,0.229898,0.374885,1.0,1.0,0.0,0.0,...,-0.017479,-0.008756,-0.009264,0.011596,0.011982,-0.005343,-0.023074,0.010134,0.014657,0.006310,-0.009259,-0.004175,-0.030011,-0.000603,0.022829,0.016007,-0.014578,0.013195,-0.001009,-0.005089,0.006830,0.003764,-0.010876,0.005384,-0.001097,0.014771,-0.003832,0.012285,-0.006332,0.000532,0.003135,-0.004830,-0.006745,0.010056,0.000014,0.000873,-0.005144,0.003438,0.003770,0.006388
1,3663550,0.004769,1.0,0.188032,0.022,0.0,1.000000,0.919708,1.0,0.452273,0.836957,0.000495,0.30,0.142857,0.001017,0.000611,0.0,0.000000,0.000000,0.000625,0.000000,0.000000,0.003497,0.000000,0.000448,0.000448,0.007682,0.002509,0.232449,0.232449,0.006506,0.581118,0.006434,0.211732,0.718007,0.581118,1.0,0.0,0.0,0.0,...,0.002133,-0.001256,-0.001551,-0.001558,-0.001137,-0.001040,0.004140,-0.000069,-0.000144,0.001253,0.000007,-0.000197,-0.001209,0.001553,-0.001969,-0.000589,0.001264,-0.000124,0.000635,0.000487,0.003265,-0.002253,0.000269,0.000628,0.000443,-0.000943,0.000235,-0.000089,0.001129,0.000476,-0.001325,-0.001971,0.000549,-0.001147,-0.000356,-0.000879,-0.000913,0.002622,-0.000467,0.000143
2,3663551,0.016649,1.0,0.199759,0.948,0.0,1.000000,0.919708,1.0,0.845455,0.836957,0.326073,0.05,0.142857,0.000678,0.000611,0.0,0.000000,0.000000,0.003123,0.000000,0.000000,0.006993,0.000000,0.000895,0.000000,0.014085,0.002509,0.213729,0.213729,0.009294,0.088909,0.009191,0.124656,0.154020,0.088909,1.0,1.0,0.0,0.0,...,-0.008925,-0.012782,0.039069,-0.016219,0.009386,-0.002430,-0.013379,-0.050106,-0.029329,0.046258,-0.007456,-0.012599,0.004748,0.047566,0.010811,-0.001997,-0.005664,-0.028481,0.003058,0.011120,0.001758,-0.000901,-0.002414,-0.004155,0.001861,0.021965,-0.009920,0.013137,-0.009509,-0.000454,-0.003418,0.008312,0.001949,0.011903,0.012673,-0.009815,-0.011519,0.007198,-0.001432,0.002093
3,3663552,0.027744,1.0,0.574155,0.520,0.0,1.000000,0.481752,1.0,0.238636,0.836957,0.002104,0.00,0.142857,0.001695,0.000611,0.0,0.000000,0.002660,0.000625,0.000000,0.000000,0.003497,0.000000,0.000895,0.000000,0.004481,0.005019,0.065523,0.065523,0.038104,0.221815,0.037684,0.221815,0.274066,0.221815,1.0,1.0,1.0,0.0,...,0.008311,-0.000749,0.005889,0.001113,-0.024552,0.000392,-0.017028,0.008433,-0.003570,-0.003047,-0.007794,-0.002740,-0.023335,-0.002066,0.035939,0.056223,-0.025821,0.005312,0.030487,-0.005495,0.027934,-0.001679,-0.010390,0.007519,-0.012180,-0.003944,-0.003549,0.006030,0.004233,-0.000221,-0.010412,0.006224,-0.003280,0.001159,0.006077,-0.002721,0.004895,0.000970,-0.006712,-0.000612
4,3663553,0.006615,1.0,0.978213,0.704,0.0,0.666667,0.124088,1.0,0.372727,0.836957,0.000742,0.00,0.142857,0.002034,0.001832,0.0,0.000000,0.005319,0.003123,0.000000,0.000000,0.008741,0.000000,0.002686,0.000000,0.008963,0.007528,0.034321,0.034321,0.000000,0.020165,0.000000,0.020165,0.024915,0.020165,1.0,1.0,1.0,0.0,...,0.001622,-0.016572,0.010184,-0.016177,-0.022733,-0.013678,-0.018686,-0.007407,0.013532,-0.003223,0.021454,-0.012770,0.007094,0.004901,-0.003330,0.006967,0.003462,0.002934,-0.019066,0.007472,-0.000021,0.006962,-0.005301,-0.002118,-0.011969,0.004248,-0.006359,-0.000693,0.003261,-0.009226,0.007046,0.001717,0.002585

In [ ]:

#x=pca.explained_variance_ratio_


In [ ]:
#Afficher le cumulé du % de var expli pour décider du nombre de composantes principale à garder, danc ce cas on garde 30 à 50 maximum 


#fig, ax =plt.subplots(figsize=(6,4))
#ax.plot( np.cumsum(x))
#ax.set(xlabel="composante", ylabel="pourcentage de variance_ration", title="variance cumulée", ylim=(0,1),)


# **Modeling**

In [ ]:
X = train.drop(columns = ['isFraud'])       # input features
Y = train.isFraud 

In [ ]:

base_tree = DecisionTreeClassifier(class_weight='balanced', random_state = 42)
base_tree.fit(X,Y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [ ]:
#base_tree_train_predict = base_tree.predict_proba(X_train)
base_tree_test_predict = base_tree.predict(test)

In [ ]:
base_tree_test_predict

In [ ]:
base_rf = RandomForestClassifier(class_weight='balanced', random_state=42)
base_rf.fit(X, Y)



In [ ]:
base_rf_test_predict = base_rf.predict(test)

In [ ]:
submission = pd.DataFrame({'TransactionID':test['TransactionID'],'isFraud':base_rf_test_predict})

#Visualize the first 5 rows
submission.head()

,TransactionID,isFraud
0,3663549,0
1,3663550,0
2,3663551,0
3,3663552,0
4,3663553,0


In [ ]:
filename = 'submission3.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: submission3.csv


### **Iterative Imputer**

In [ ]:
#from sklearn.impute import KNNImputer
#def impute_DM_Imp(X):
#  imputer = KNNImputer(n_neighbors=2)
#  imputer.fit_transform(X)
#  return (X)
#train= impute_DM_Imp(train)


In [ ]:
#from sklearn.ensemble import ExtraTreesRegressor
#def impute_DM_Iterative_Imp(df_train_complete,df_test_complete): # use iterative Imputer to impute the Nan of the rest of the

    #isFraud= df_train_complete["isFraud"]   
    #df_train_complete.drop(columns={"isFraud"}, index=1, inplace=True)                                                       
    
    #imputer = IterativeImputer(estimator= ExtraTreesRegressor(), n_nearest_features=None, imputation_order='ascending')
  
    #imputer.fit(df_train_complete)
   
    #df_train_complete = imputer.transform(df_train_complete)

    #df_test_complete = imputer.transform(df_test_complete)

    #df_train_complete["isFraud"]=isFraud
    #return (train,test)


#train, test = impute_DM_Iterative_Imp(train.iloc[0:3000,0:10],test.iloc[0:3000,0:10])

 

In [ ]:
#def LabelEncode(train,test):
 # encoders = dict() ##La fonction dict () crée un dictionnaire.
 # cat_cols = train.select_dtypes(include = 'object').columns
 # for col_name in cat_cols :
 #   series_train = train[col_name]
 #   series_test = test[col_name]

  #  label_encoder = LabelEncoder()
 #   label_encoder.fit(list(series_train[series_train.notnull()].astype(str).values) + list(series_test[series_test.notnull()].astype(str).values))
        
  #  train[col_name] = pd.Series(
  #  label_encoder.transform(series_train[series_train.notnull()]),#series [series.notnull ()] supprime les valeurs NaN, puis transmet le reste à fit_transform.
  #  index=series_train[series_train.notnull()].index
  #  )
  #  test[col_name] = pd.Series(
  #  label_encoder.transform(series_test[series_test.notnull()]),#series [series.notnull ()] supprime les valeurs NaN, puis transmet le reste à fit_transform.
  #  index=series_test[series_test.notnull()].index
  #  )
  #return (train, test)

#train, test=LabelEncode(train,test)

In [ ]:
# Reducing memory usage
# From https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
# WARNING! THIS CAN DAMAGE THE DATA
def reduce_mem_usage2(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [ ]:
train = reduce_mem_usage2(train)
test = reduce_mem_usage2(test) 

In [ ]:
train.drop(train[train.dist1>6000].index, axis=0, inplace=True)
train.drop(train[train.TransactionAmt>10000].index, axis=0, inplace=True)
train.loc[train.C1 > 3000, "C1" ] = train.loc[train.C1 <= 3000, "C1" ].mean()
train.loc[train.C2 > 3000, "C2" ] = train.loc[train.C2 <= 3000, "C2" ].mean()
train.loc[train.C4 > 1400, "C4" ] = train.loc[train.C4 <= 1400, "C4" ].mean()
train.loc[train.C6 > 1600, "C6" ] = train.loc[train.C6 <= 1600, "C6" ].mean()
train.loc[train.C7 > 1400, "C7" ] = train.loc[train.C7 <= 1400, "C7" ].mean()
train.loc[train.C8 > 1000, "C8" ] = train.loc[train.C8 <= 1000, "C8" ].mean()
train.loc[train.C10 > 2000, "C10" ] = train.loc[train.C10 <= 2000, "C10" ].mean()
train.loc[train.C11 > 2000, "C11" ] = train.loc[train.C11 <= 2000, "C11" ].mean()
train.loc[train.C12 > 2000, "C12" ] = train.loc[train.C12 <= 2000, "C12" ].mean()
train.loc[train.C13 > 1200, "C13" ] = train.loc[train.C13 <= 1200, "C13" ].mean()
train.loc[train.C14 > 800, "C14" ] = train.loc[train.C14 <= 800, "C14" ].mean()